In [2]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd
from packages.csv_to_df import to_df

In [3]:
# Los datos extraídos se almacenan en una lista plana, por lo que hay que transformarla en un dataframe
def create_df(flat_list):
    # Combina todas las sublistas en una única lista
    flattened_data = [row for sublist in flat_list for row in sublist]
    
    # Extrae los encabezados de las columnas y las filas de datos
    column_headers = flattened_data[0]
    data_rows = flattened_data[1:]
    
    # Crea un DataFrame a partir de las filas de datos y los encabezados de las columnas
    df = pd.DataFrame(data_rows, columns=column_headers)
    
    # Elimina las filas duplicadas en el DataFrame
    df.drop_duplicates(inplace=True)
    
    return df

In [4]:
# Agregar errores que se hacen cuando se crea el dataframe
def fix_df(df):
    # Definir la primera fila con los nombres de las columnas correctos
    first_row = ['Fecha', 'Ronda', 'Equipo Local', 'Resultado Local', 'Equipo Visitante', 'Resultado Visitante', 'Cancha', 'Condición', 'URL', 'Competición']
    
    # Asignar los nombres de las columnas al DataFrame
    df.columns = first_row
    
    # Restablecer los índices del DataFrame para que comiencen desde 0
    df.reset_index(drop=True, inplace=True)
    
    return df

In [5]:
def save_csv(df, filename):
    # Guardar el DataFrame como un archivo CSV con el nombre especificado
    df.to_csv(f'{filename}.csv', sep=';')
    
    return df

In [6]:
# Usando el URL de la página de Historia de Boca consigue el nombre de la competición que se está analizando
def get_competition(url):
    # Dividir la URL en partes usando '/' como delimitador
    url_splited = url.split('/')
    
    # Obtener el segmento de la URL que contiene el nombre de la competición
    competition_kebab = url_splited[3]
    
    # Dividir el nombre de la competición en palabras y convertirlas en mayúsculas
    competition_list = competition_kebab.split('-')
    competition = ' '.join(word.capitalize() for word in competition_list)
    
    return competition

In [7]:
# Usa la página de Wikipedia para obtener los datos de qué años Boca disputó una competencia internacional especificada
def get_participation(competition):
    # URL de la página de Wikipedia
    url = 'https://es.wikipedia.org/wiki/Anexo:Boca_Juniors_en_competiciones_internacionales'
    
    # Realizar la solicitud HTTP GET a la URL
    response = requests.get(url)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        
        # Parsear el contenido HTML de la página web
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar todas las tablas con la clase 'sortable'
        tables = soup.find_all('table', class_='sortable')
        
        # Lista para almacenar los datos
        data_list = []
        
        # Iterar sobre las tablas encontradas
        for table in tables:
            # Encontrar todas las filas de la tabla
            rows = table.find_all('tr')
            # Iterar sobre las filas
            for row in rows:  
                # Encontrar todas las celdas de la fila
                cells = row.find_all('td')
                # Iterar sobre las celdas
                for cell in cells:
                    # Encontrar todos los enlaces dentro de la celda
                    links = cell.find_all('a')
                    # Iterar sobre los enlaces
                    for link in links:
                        # Verificar si el texto del enlace comienza con el nombre de la competición
                        if link.text.strip().startswith(competition):
                            # Extraer el año de la participación y agregarlo a la lista de datos
                            competition_year = link.text.strip().replace(competition, '').strip()
                            data_list.append(competition_year)
                            
        return data_list
    else: 
        print('Error al cargar la página:', response.status_code)
        
        return None

In [8]:
# Obtener el valor de los años donde se jugó una competición para luego usarlo en la función que extrae los datos
def get_year_range(inicio, fin):
    lista_numeros = range(inicio, fin + 1)
    return lista_numeros

In [9]:
# Divide el texto de una celda en la tabla de partidos para hacer que el equipo local, visitante y sus resultados estén en columnas distintas
def split_match(text):
    elements = text.split(' - ')
    elements_splited = []
    pattern = re.compile(r'(.+?)\s*(\d+\s*(?:\(\d+\))?)$')
    
    for element in elements:
        coincidence = pattern.match(element)
        
        if coincidence:
            elements_splited.append(coincidence.group(1).strip())
            elements_splited.append(coincidence.group(2).strip())
        else:
            elements_splited.append(element.strip())
    
    return elements_splited

In [10]:
# Scrapea la tabla con los partidos de la URL de la campaña de Boca en una competición y año específico
def scrap_campain(url):
    response = requests.get(url)
    
    competition = get_competition(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('table', class_='listaPartidos')
        
        if tables:
            table_data = []
            
            for table in tables:
                for div in table.find_all('div'):
                   div.decompose()
                    
                for i, row in enumerate(table.find_all('tr')[1:]):
                    rows_data = []
                    columns = row.find_all('td')
                    
                    for column in columns:
                        match_link = column.find('a')
                        
                        if match_link:
                            if 'fontnegra' in match_link.get('class', []):
                                lista = match_link.text.strip()
                                match = split_match(lista)
                                for item in match:
                                    rows_data.append(item)
                            else:
                                rows_data.append(match_link.get('href'))
                        else: 
                            text = column.text.strip()
                            try:
                                date = pd.to_datetime(text, format='%d/%m/%Y').strftime('%Y-%m-%d')
                                rows_data.append(date)
                            except ValueError:
                                rows_data.append(text)
                    if i == 0:
                        first_row = rows_data
                    rows_data.append(competition)
                    table_data.append(rows_data)
            if first_row:
                table_data.insert(0, first_row)
            return table_data
        
    else: return f'Error {response.status_code}'

In [11]:
# Scrapea todos los partidos de Boca en la competición internacional que le proporcionemos
def scrap_int_competition(search_for, competition, number_range):
    competition_list = []
    participations = get_participation(search_for)
    for year in participations:
        for number in range(number_range[0], number_range[1]+1):
            url = f'https://www.historiadeboca.com.ar/{competition}-{year}/{year}/{number}.html'
            response = requests.get(url)
            if response.status_code == 200:
                df = scrap_campain(url)
                competition_list.append(df)
                break
            else: continue
    return competition_list

In [12]:
# Scrapea todos los partidos de Boca en la competición nacional que le proporcionemos
def scrap_nat_competition(inicio, fin, competition, number_range):
    competition_list = []
    years = get_year_range(inicio, fin)
    for year in years:
        for number in range(number_range[0], number_range[1]+1):
            url = f'https://www.historiadeboca.com.ar/{competition}-{year}/{year}/{number}.html'
            response = requests.get(url)
            if response.status_code == 200:
                df = scrap_campain(url)
                competition_list.append(df)
                break
            else: continue
    return competition_list

Recopilar, crear DataFrame y guardarlo con todos los partidos disputados de Boca en una competición internacional

In [ ]:
# Competición escrita según como está en la URL de las campañas en HistoriaDeBoca.com.ar
competition = 'libertadores'
# Buscar todas las participaciones de Boca en la competición internacional que le digamos, debe coincidir en cómo está escrita en la página de Wikipedia
search_for = 'Copa Libertadores' + ' '
# Nombre de cómo guardaremos el archivo
name = 'Liberadores'
# Último valor de la URL de una campaña de Boca, debemos fijarnos en este valor para la primera y última vez que Boca disputó la competencia 
# (Aunque a veces debamos agregar más valores ya que no siempre es exacto)
number_range = [195, 210]

# Recopila los datos en una lista, la convierte en un dataframe, arregla imperfecciones, lo guarda con el nombre especificado y se muestra
list = scrap_int_competition(search_for, competition, number_range)
df = create_df(list)
df = fix_df(df)
df = save_csv(df, f'dataframes_competitions/{name}')
df

Recopilar, crear DataFrame y guardarlo con todos los partidos disputados de Boca en una competición nacional

In [20]:
# Competición escrita según como está en la URL de las campañas en HistoriaDeBoca.com.ar
competition = 'amistosos'
# Rango de años donde Boca jugó por primera y última vez la competición
inicio, fin = 2001, 2024
# Nombre de cómo guardaremos el archivo
name = 'Amistosos'
# Último valor de la URL de una campaña de Boca, debemos fijarnos en este valor para la primera y última vez que Boca disputó la competencia 
# (Aunque a veces debamos agregar más valores ya que no siempre es exacto)
# (En este caso se recomienda hacerlos por partes ya que este valor varía mucho y de lo contrario el proceso de recopilar los datos sería muy largo y pesado)
number_range = [2101, 2124]

# Recopila los datos en una lista, la convierte en un dataframe, arregla imperfecciones, lo guarda con el nombre especificado y se muestra
list = scrap_nat_competition(inicio, fin, competition, number_range)
df = create_df(list)
df = fix_df(df)
df = save_csv(df, f'amistosos/{name}')

Convertir todos los DataFrames de las distintas competiciones en uno solo, además de arreglar imperfecciones

In [21]:
# Carpeta donde están almacenados los DataFrames
folder = 'amistosos'

# Nombre que llevará el archivo cuando lo guardemos
filename = 'Amistosos_Boca_DF'

# Recopilar todos los archivos con extensión ".csv"
csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]

dataframes = []

# Agregar todos los DataFrames
for file in csv_files:
    file_path = os.path.join(folder, file)
    df = pd.read_csv(file_path, sep=';')
    df.drop(columns=['Unnamed: 0'], inplace=True)
    dataframes.append(df)
    
# Arreglar imperfecciones y guardarlo en formato ".csv"
df_final = pd.concat(dataframes, ignore_index=True)
df_final = df_final.sort_values(by='Fecha')
df_final = df_final.reset_index(drop=True)
save_csv(df_final, filename)

,Fecha,Ronda,Equipo Local,Resultado Local,Equipo Visitante,Resultado Visitante,Cancha,Condición,URL,Competición
0,1905-04-21,Amistoso,Boca,4,Mariano Moreno,0,Boca Juniors,L,partido/boca-4-mariano-moreno-0-amistosos-1905...,Amistosos 1905
1,1905-04-30,Amistoso,Boca,2,Presidente Roca,0,Boca Juniors,L,partido/boca-2-presidente-roca-0-amistosos-190...,Amistosos 1905
2,1905-05-07,Amistoso,Boca,1,California,3,Boca Juniors,L,partido/boca-1-california-3-amistosos-1905/514...,Amistosos 1905
3,1905-05-14,Amistoso,Radical,7,Boca,1,Radical,V,partido/radical-7-boca-1-amistosos-1905/5141/1...,Amistosos 1905
4,1905-06-04,Amistoso,Mariano Moreno,4,Boca,0,Colegio Nacional Sud,V,partido/mariano-moreno-4-boca-0-amistosos-1905...,Amistosos 1905
...,...,...,...,...,...,...,...,...,...,...
1450,2022-01-25,Final. Partido Único,Boca,1,San Lorenzo,0,Estudiantes (La Plata),N,partido/boca-1-san-lorenzo-0-amistosos-2022/63...,Amistosos 2022
1451,2023-01-07,Partido 1,Boca,0,Independiente,0,Bicentenario (San Juan),N,partido/boca-0-independiente-0-amistosos-2023/...,Amistosos 2023
1452,2023-01-13,Partido 2,Boca,0,Everton (Chile),0,Bicentenario (San Juan),L,partido/boca-0-everton-chile-0-amistosos-2023/...,Amistosos 2023
1453,2024-01-13,Amistoso,Gimnasia y Tiro (S),0,Boca,1,Padre Ernesto Martearena (Salta),V,partido/gimnasia-y-tiro-s-0-boca-1-amistosos-2...,Amistosos 2024
